In [ ]:
import requests
from pymongo import MongoClient
from urllib.parse import urljoin
from pprint import pprint
import nltk
import json

# Retrieve Lyrics Via API

In [ ]:
# DB CONFIG
client = MongoClient('mongo', 27017)
db = client.music_db
COLLECTION = 'rap-songs'
songs_collection = db[COLLECTION]

In [ ]:
# Pull all songs from DB
songs = list(songs_collection.find())

In [ ]:
MUSICMATCH_API = '309401c7a9740d7c3bfbc94bca9a911c'
APISEED_API = 'ICmmIMXiTjyqhQjXC0HX4oqZP3vEOp9zr2prrrM5MUDe4hWri6gVTCebOCkARWi0'

In [ ]:
class ApiSeed:
    BASE_URL = 'https://orion.apiseeds.com/api/music/lyric/'
    
    def __init__(self, api_key):
        """ Load automatically translates responses from HTTP request objects into Dictionaries """
        self.api_key = api_key
        self.load = True
    
    def _api_url(self, url):
        url = urljoin(self.BASE_URL, url)
        url = urljoin(url, '?apikey=ICmmIMXiTjyqhQjXC0HX4oqZP3vEOp9zr2prrrM5MUDe4hWri6gVTCebOCkARWi0')
        return url
    
    def _retrieve(self, url):
        response = requests.get(self._api_url(url))
        if self.load:
            response = json.loads(response.content)
        return response
    
    def get_track(self, artist, track):
        return self._retrieve(f'{artist}/{track}')
    

apiseed = ApiSeed(APISEED_API)
res = apiseed.get_track('Kanye West', 'Everything I am')

# Insert Lyrics into DB

In [9]:
import time
def insert_into_db():
    for song in songs:
        pass
    
def update_db(document_id, lyrics: str):
    document = songs_collection.find_one_and_update({"_id": document_id}, 
                                         {"$set": {"lyrics": lyrics}})
    return document

def retrieve_lyrics(track: dict):
    """ Takes mongodb document and gets lyrics"""
    res = apiseed.get_track(track['artist'], track['title'])
    return res

def find_and_update_lyrics(track: dict):
    """ Updates track with lyrics """
    lyrics = retrieve_lyrics(track)
    track['lyrics'] = lyrics
    return track

        
start_index = 0
all_tracks = []
for i, track in enumerate(songs[start_index:], start_index):
    try:
        track = find_and_update_lyrics(track)

        if track.get('lyrics').get('error'):
            print("Error", end=" "); 
        else:
            print("Success", end=" ")
            
    except (json.JSONDecodeError, Exception) as e:
        print(e)
        track['lyrics'] = 'null'
        
    update_db(track['_id'], track['lyrics'])
    print(i, track['title'], end = " | ")
    time.sleep(.33)



Error 0 Meant To Be | Success 1 Tequila | Success 2 Simple | Success 3 Heaven | Success 4 Hotel Key | Error 5 She Got The Best Of Me | Success 6 Sunrise, Sunburn, Sunset | Success 7 Blue Tacoma | Success 8 Hooked | Success 9 Lose It | Success 10 Break Up In The End | Success 11 Drowns The Whiskey | Success 12 Drunk Me | Success 13 Get Along | Success 14 Drunk Girl | Error 15 Speechless | Error 16 Desperate Man | Success 17 Hangin' On | Success 18 Best Shot | Success 19 Rich | Success 20 Born To Love You | Error 21 Beautiful Crazy | Error 22 Burning Man | Success 23 Hide The Wine | Error 24 Babe | Success 25 Love Wins | Error 26 All Day Long | Success 27 Last Shot | Success 28 Millionaire | Success 29 Turnin' Me On | Success 30 Good Girl | Success 31 Here Tonight | Success 32 Burn Out | Success 33 Take It From Me | Success 34 Down To The Honkytonk | Success 35 This Is It | Error 36 Here I Am | Error 37 Homesick | Error 38 There Was This Girl | Error 39 David Ashley Parker From Powder Sp

In [ ]:
songs_collection = db[COLLECTION]

def has_lyrics(song):
    if not song.get('lyrics'):
        return False
    
    if song.get('lyrics') == 'null':
        return False

    if song.get('lyrics').get('error'):
        return False
        
    return True


In [ ]:
songs = [song for song in songs_collection.find() if has_lyrics(song)]

In [ ]:
len(songs)